In [31]:
import pandas as pd
import numpy as np

def forma(x):
    x = [float(i.replace(',','.')) if i != 'N' else 'N' for i in x.values]
    return x

data_discover = pd.read_csv(r"Discover Часовые общий.csv", sep=";")

names = ['bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 'proton_density', 'proton_speed', 'proton_temperature']

for elem in names:
    data = forma(data_discover[elem])
    data_discover[elem] = data

In [32]:
data_discover.iloc[range(1256,1264), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.2459
1257,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1258,N,N,N,N,N,N
1259,N,N,N,N,N,N
1260,N,N,N,N,N,N
1261,N,N,N,N,N,N
1262,6.326842,3.803684,3.412105,3.584211,N,N
1263,5.303443,3.611967,-2.94918,2.031148,12.658125,210951.0313


In [33]:
pd.set_option('future.no_silent_downcasting', True)

for elem in names:
    counter, out = -1, 0
    for index, value in data_discover[elem].items():
        if (value == 'N') & (counter < 12):
            counter += 1
        elif (value == 'N') & (counter >= 12):
            counter = 0
            out = 1
        elif (value != 'N') & (counter in range(1, 13)) & (out == 0):
            data_discover.loc[range(index-counter, index+1), elem] = np.NaN
            data_slice = data_discover.loc[range(index-counter-1, index+1), elem]
            data_slice_obj = data_slice.infer_objects(copy=False)
            data_discover.loc[range(index-counter-1, index+1), elem] = data_slice_obj.interpolate(method='linear')
            counter = 0
        else:
            counter = 0
            out = 0


# data_discover.to_csv("Discover с интреполяцией12.csv", sep=';', index=False, encoding='utf-8')

In [34]:
data_discover.iloc[range(1256,1264), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.2459
1257,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1258,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1259,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1260,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1261,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1262,7.372292,-1.107917,-6.313542,2.603542,17.14,94458.79167
1263,5.303443,3.611967,-2.94918,2.031148,17.14,94458.79167
